<a href="https://colab.research.google.com/github/rjscherrer/reinforcement-learning-tasks/blob/main/chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>